# Lab 4 – LoRA Fine-Tuning
**Part 4 of the 7 Lab Hands-On SLM Training Series**

In this lab, we move from preparing our dataset to teaching the Small Language Model (SLM) how to adapt to our domain. We do this using **LoRA (Low-Rank Adaptation)**, a parameter-efficient fine-tuning technique supported by the `unsloth` library.

By the end of this lab you will have:
- Attached LoRA adapters to a base model
- Run a short domain-adaptive training loop
- Verified that the model is adapting to your dataset

## Step 0. Install dependencies

In [3]:
!pip install -q unsloth transformers datasets accelerate bitsandbytes
# Make Lab 4 compatible with datasets saved using "List" features
%pip install -q --force-reinstall "numpy==2.0.2" "pandas==2.2.2" "pyarrow==17.0.0"
%pip install -q "datasets>=3.0.0" "transformers>=4.41.0" "sentencepiece>=0.1.99" "tqdm>=4.66.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 30.8 MB/s eta 0:00:00


## Step 1. Load the prepared dataset from Google Drive

In [4]:
from datasets import load_from_disk
from google.colab import drive
drive.mount('/content/drive')

# Load the tokenized dataset created in Lab 3 (saved to Google Drive)
DATA_DIR = "/content/drive/MyDrive/slm-labs/lab3_tokenized"
dataset = load_from_disk(DATA_DIR)
print(dataset)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: Feature type 'List' not found. Available feature types: ['Value', 'ClassLabel', 'Translation', 'TranslationVariableLanguages', 'LargeList', 'Sequence', 'Array2D', 'Array3D', 'Array4D', 'Array5D', 'Audio', 'Image', 'Video', 'Pdf']

## Step 2. Load the base model

In [ ]:
from unsloth import FastLanguageModel

model_name = "HuggingFaceH4/zephyr-7b-beta"  # replace with your base SLM if preferred

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model and tokenizer ready.")

## Step 3. Attach LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)
print("LoRA adapters attached.")

## Step 4. Fine-tune with LoRA

In [ ]:
from transformers import TrainingArguments
from unsloth import UnslothTrainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=50,  # keep small for demo
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    save_strategy="no",
    output_dir="./outputs",
)

trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset.get("test"),
)

trainer.train()

## Step 5. Save LoRA adapters

In [ ]:
model.save_pretrained("./lora_adapters")
tokenizer.save_pretrained("./lora_adapters")
print("LoRA adapters saved to ./lora_adapters")

### Wrap-up
In this lab you have:
- Loaded your domain dataset from Lab 3 (via Google Drive)
- Attached LoRA adapters to your base model
- Run a short fine-tuning loop
- Saved the resulting adapters

These adapters can now be reapplied to the base model any time you want, making fine-tuning lightweight and reusable.

Next up: **Lab 5 – Hyperparameter Tuning and Optimization**.